In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL
import json
import textwrap
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


## Paeds

In [ ]:
# workbookpath = '2024\DCD\Postgraduate Clinical Evaluation Paeds_March 14, 2025_13.24.xlsx'
# workbookpath = '2025\DCD\Paeds\Postgraduate Clinical Evaluation Paeds - 2025_June 23, 2025_09.58.xlsx'
# workbookpath = '2025\DCD\Oral Med\Postgraduate Clinical Evaluation - DCD in Oral Medicine_May 15, 2025_14.31.xlsx'
workbookpath =  '2025\DCD\Prostho\Postgraduate Clinical Evaluation - DCD in Prosthodontics_July 1, 2025_14.01.xlsx'
# workbookpath = '2025\DCD\Paeds\Postgraduate Clinical Evaluation Paeds - 2025_June 26, 2025_09.59.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
# mccols are with Q\d+_\d+ pattern
mcCols = [col for col in df.columns if re.match(r'Q\d+_\d+', col)]
mcColIdentifier = 'Q19'
# remove 
generalCols = [col for col in mcCols if mcColIdentifier in col]
mcCols = [col for col in mcCols if mcColIdentifier not in col]
mcCols = [col for col in mcCols if 'TEXT' not in col]  # remove columns with TEXT in the name
print(mcCols)
colDate = 'Date'
extraCols = ['Time management', 'Entrustment scale']
# competency scale columns
colComp = 'Q18'
colorlegendcomp = {'Yes': '#4c72b0', 'No': '#e07a5f', 'Not applicable': 'gray'}
# comment columns are Q20 and Q21
commentCols =[ 'Q20', 'Q21']
finalgradedict = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':5, 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':4, 
                  'At expected level of performance considering stage of training: H3 (65-69)':3, 'Barely meets expected level of performance considering stage of training: P (50-64)':2,
                  'Does not meet expected level of performance considering stage of training: N (0-49)':1, 'No basis for evaluation':np.nan, 'Clinical rotation not completed':np.nan, 'Not applicable':np.nan,
                  'Exceeds expected level of performance considering stage of training': 4, 'At expected level of performance considering stage of training': 3, 'Barely meets expected level of performance considering stage of training':2}
reversefinalgradedict = {5: 'Far exceeds expected level of performance considering stage of training', 4: 'Exceeds expected level of performance considering stage of training',
                         3: 'At expected level of performance considering stage of training', 2: 'Barely meets expected level of performance considering stage of training', 1: 'Does not meet expected level of performance considering stage of training',
                         np.nan: 'No basis for evaluation'}

colorlegendfinal = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':'#4c72b0', 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':'#55a868',
                    'At expected level of performance considering stage of training: H3 (65-69)':'#e07a5f', 'Barely meets expected level of performance considering stage of training: P (50-64)':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training: N (0-49)':'#993f14', 'Clinical rotation not completed':'gray', 
                    'Far exceeds expected level of performance considering stage of training':'#4c72b0',
                    'Exceeds expected level of performance considering stage of training':'#55a868', 'At expected level of performance considering stage of training':'#e07a5f', 'Barely meets expected level of performance considering stage of training':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training':'#993f14', 'No basis for evaluation':'gray', 'Not applicable':'gray'}
colFinalGrade = 'Q22'
df[colFinalGrade] = df[colFinalGrade].replace(finalgradedict)
df[colFinalGrade] = df[colFinalGrade].astype(float)
mcgradedict = {'Completed with substantial assistance':1, 'Completed with minimal assistance':2, 'Completed independently (competent)':3, 'No basis for evaluation': np.nan}
generalgradedict = {'ALWAYS': 4, 'OFTEN': 3, 'SOMETIMES': 2, 'RARELY': 1, 'NEVER': 0, 'Not graded': np.nan}
# create color legend dictionary
colorlegendmc = {'Completed with substantial assistance': '#e07a5f', 'Completed with minimal assistance': '#55a868', 'Completed independently (competent)': '#4c72b0', 'No basis for evaluation': 'gray',
              }
colorlegendgeneral =  {'ALWAYS': '#4c72b0', 'OFTEN': '#55a868', 'SOMETIMES': '#e07a5f', 'RARELY': '#f9c74f', 'NEVER': '#993f14', 'Not graded': 'gray'}
reversemcgradedict = {v:k for k, v in mcgradedict.items()}
referencedf = pd.DataFrame(reversemcgradedict.items(), columns=['Grade', 'Description'])
# drop nan values from grade column
referencedf.dropna(inplace=True)
referencedf['Grade'] = referencedf['Grade'].astype(int)
referencedf.sort_values(by='Grade', inplace=True)
referencetable = createTable(referencedf, 'Score reference', [1, 5], customTextCols=[1])
df[mcCols] = df[mcCols].replace(mcgradedict)
df[colDate] = pd.to_datetime(df[colDate], format = 'mixed')
df.sort_values(by=[colName, colDate], inplace=True)
# take only may and june 2025 dates
# df = df[(df[colDate].dt.year == 2025) & (df[colDate].dt.month.isin([5, 6]))]
display(df.head())
df[colDate] = df[colDate].dt.date
df[colDate] = df[colDate].astype(str)
df[mcCols] = df[mcCols].astype(float)

# count the number of each scores for each student
df['#3'] = df[mcCols].apply(lambda x: x.value_counts().get(3, 0), axis=1)
df['#2'] = df[mcCols].apply(lambda x: x.value_counts().get(2, 0), axis=1)
df['#1'] = df[mcCols].apply(lambda x: x.value_counts().get(1, 0), axis=1)
df['#nan'] = df[mcCols].apply(lambda x: pd.isna(x)).sum(axis=1)

df[generalCols] = df[generalCols].replace(generalgradedict)
# display(df.head())

students = df[colName].unique()

def chunkList(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


def createPieCharts(mcCols, n, mcgradedict, colorlegend, elements, studentdf, fullnamedict):
    for groupIndex, mcChunk in enumerate(chunkList(mcCols, n)):
        fig, axes = plt.subplots(3, 2, figsize=(figSize[0], figSize[1]))
        fig.subplots_adjust(hspace=0.4, wspace=0.3)
        axes = axes.flatten()
        for idx, col in enumerate(mcChunk):
            ax = axes[idx]
            yTitle = 1.3
            faceColor = '#f2f2f2'  # background shading
            # add frame
            framePadding = 0.02
            bbox = ax.get_position()
            # Calculate extended box
            x0 = bbox.x0 - 2 * framePadding
            y0 = bbox.y0 - framePadding
            width = bbox.width + 4 * framePadding
            height = bbox.height + (yTitle - 1.23) + framePadding  # 1.0 is top of axes

            # Draw rectangle in figure space
            rect = Rectangle(
                (x0, y0), width, height,
                linewidth=1,
                edgecolor='black',
                facecolor=faceColor,
                transform=fig.transFigure,
                zorder=-5
            )
            fig.patches.append(rect)


            valuecounts = studentdf[col].value_counts(dropna=False)   
            display(valuecounts) 
            newkeys = []
            for v in valuecounts.index:
                for k, val in mcgradedict.items():
                    if (pd.isna(v) and pd.isna(val)) or (v == val):
                        newkeys.append(k)
                        break
            print(newkeys)

            valuecounts.index = newkeys
            total = valuecounts.sum()
            display(valuecounts)
            colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
            
            # create pie without labels
            wedges, _, autotexts = ax.pie(
                valuecounts,
                labels=None,
                autopct=lambda x: autopct(x, total),
                colors=colors,
                textprops={'fontsize': 7}
            )
            

            title = textwrap.fill(fullnamedict[col].split(' - ')[-1], width=55)  # Adjust width as needed
            # ax.set_title(title, fontsize=9)
            ax.text(0.5, yTitle, title, ha='center', va='top', fontsize=8, transform=ax.transAxes, wrap=True)


            # Add legend to bottom-left of figure
            legendHandles = [
                plt.Line2D([0], [0], marker='o', color='w', label=label,
                        markerfacecolor=color, markersize=8)
                for label, color in colorlegend.items()
            ]

            fig.legend(
                handles=legendHandles,
                loc='upper left',
                bbox_to_anchor=(0.09, 1.03),
                fontsize=8
            )

        # Hide unused subplots
        for j in range(len(mcChunk), n):
            axes[j].axis('off')


        # Save or display
        img = addPlotImage(fig, 0.9)
        # elements.append(KeepTogether([Paragraph(f"Batch {groupIndex+1} Pie Charts", smallsubsubheadingStyle), Spacer(1, 12), img]))
        elements.append(KeepTogether([img]))
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    # elements.append(PageBreak())
def getReportType2(student): # only plot the pie charts for each mc question, general question and final grade
    elements = []
    elements.append(Paragraph(f'{student}', style = subheadingStyle))
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 24))
    # plot mc question ignoring the nan values
    n = 6
    createPieCharts(mcCols, n, mcgradedict, colorlegendmc, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot general question ignoring the nan values
    createPieCharts(generalCols, n, generalgradedict, colorlegendgeneral, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # add pie chart for competency scale ------------------------------------------------------------------------
    fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    xy = studentdf[colComp].dropna()
    xy = xy.value_counts()
    total = xy.sum()
    xy = xy.sort_index()
    ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = xy.index, colors=[colorlegendcomp.get(k, 'gray') for k in xy.index])
    title = "Competent to work semi-independently at Royal Children's Hospital"
    ax.set_title(None)
    img = addPlotImage(fig, 0.3)
    elements.append(KeepTogether([Paragraph(title, style = subsubheadingStyle), Spacer(1, 12), img]))
    plt.close(fig)
    elements.append(Spacer(1, 24))

    # plot final grade ignoring the nan values ----------------------------------------------------------------------
    elements.append(Paragraph('Final Grades', style = subsubheadingStyle))
    elements.append(Spacer(1, 24))
    refdf2 = pd.DataFrame(reversefinalgradedict.items(), columns=['Grade', 'Description'])
    refdf2.dropna(inplace=True)
    refdf2['Grade'] = refdf2['Grade'].astype(int)
    # drop nan values from grade column

    refdf2.sort_values(by='Grade', inplace=True)
    refTable2 = createTable(refdf2, 'Final Grade Reference', [1, 5], customTextCols=[1])
    # elements.append(refTable2)
    # elements.append(Spacer(1, 24))
    # plot time series of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0], figSize[1]/5))
    xy = studentdf[[colDate, colFinalGrade]].dropna()
    xy = xy.groupby(colDate).mean().reset_index()     # aggregate the scores for each date by taking the mean
    ax.plot(xy[colDate], xy[colFinalGrade])
    title = 'Average Time Series'
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=6)
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))     # integer ticks on y axis
    ax.set_ylim(0, 5.4)
    ax.set_title(title, fontsize=10)
    plt.grid('--', alpha=0.5)
    img = addPlotImage(fig, 0.9)
    # elements.append(img)
    plt.close(fig)
    # elements.append(Spacer(1, 24))

    # pie chart of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    xy = studentdf[colFinalGrade].dropna()
    xy = xy.value_counts()
    total = xy.sum()
    xy = xy.sort_index()
    xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
    labels = [textwrap.fill(k, width=100) for k in xy.index]
    ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels, colors=[colorlegendfinal.get(k, 'gray') for k in xy.index])
    # Add legend to bottom-left of figure
    # legendHandles = [
    #     plt.Line2D([0], [0], marker='o', color='w', label=label,
    #             markerfacecolor=color, markersize=6)
    #     for label, color in colorlegendfinal.items()
    # ]

    # fig.legend(
    #     handles=legendHandles,
    #     loc='upper left',
    #     bbox_to_anchor=(0.09, 0.03),
    #     fontsize=6
    # )
    title = 'Final Grade Distribution'
    ax.set_title(None)
    img = addPlotImage(fig, 0.9)
    elements.append(img)
    plt.close(fig)
    elements.append(Spacer(1, 24))
    
    # # create pie charts for extra columns similar to final grades
    # for col in extraCols:
    #     fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    #     xy = studentdf[col].dropna()
    #     xy = xy.value_counts()
    #     total = xy.sum()
    #     xy = xy.sort_index()
    #     # xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
    #     labels = [textwrap.fill(k, width=100) for k in xy.index]
    #     ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels)
    #     title = f'{col} Distribution'
    #     ax.set_title(None)
    #     img = addPlotImage(fig, 0.9)
    #     elements.append(KeepTogether([Paragraph(title, style = subheadingStyle), Spacer(1, 12), img]))
    #     plt.close(fig)
    #     elements.append(Spacer(1, 24))
    
    # add comments of Q20 and Q21 as table
    for col in commentCols:
        commentsDf = studentdf[['Staff', col]].dropna()
        commentsDf.rename(columns={col: 'Comments'}, inplace=True)
        display(commentsDf.head())
        
        table = createTable(
            commentsDf,
            f'{fullnamedict[col]}',
            [1, 5],
            customTextCols=[0, 1],
            titleStyle=variableUtils.smallsubsubheadingStyleC,
            tableTextStyle=tableTextStyleSmall
        )
        
        elements.append(table)
        elements.append(Spacer(1, 24))
    # elements.append(Spacer(1, 24))

    savefolder = f'{folder}/studentReports Brief'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)
students = df[colName].unique()
for student in students:
    getReportType2(student)
    # break
# getReport('Grace Wu')
# getReport('Andrew Park')

## Prostho

In [ ]:
workbookpath =  '2025\DCD\Prostho\Postgraduate Clinical Evaluation - DCD in Prosthodontics_July 1, 2025_14.01.xlsx'

folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
# mccols are with Q\d+_\d+ pattern
mcCols = [col for col in df.columns if re.match(r'Q\d+_\d+', col)]
mcColIdentifier = 'Q9'
# remove 
generalCols = [col for col in mcCols if mcColIdentifier in col]
mcCols = [col for col in mcCols if mcColIdentifier not in col]
mcCols = [col for col in mcCols if 'TEXT' not in col]  # remove columns with TEXT in the name
print(mcCols)
colDate = 'Date'
extraCols = ['Time management', 'Entrustment scale']

# comment columns are Q10
commentCols =[ 'Q10']

# critical error columns
colCI = 'Critical incident'
colCIComment = 'CI Details'

finalgradedict = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':5, 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':4, 
                  'At expected level of performance considering stage of training: H3 (65-69)':3, 'Barely meets expected level of performance considering stage of training: P (50-64)':2,
                  'Does not meet expected level of performance considering stage of training: N (0-49)':1, 'No basis for evaluation':np.nan, 'Clinical rotation not completed':np.nan, 'Not applicable':np.nan,
                  'Exceeds expected level of performance considering stage of training': 4, 'At expected level of performance considering stage of training': 3, 'Barely meets expected level of performance considering stage of training':2}
reversefinalgradedict = {5: 'Far exceeds expected level of performance considering stage of training', 4: 'Exceeds expected level of performance considering stage of training',
                         3: 'At expected level of performance considering stage of training', 2: 'Barely meets expected level of performance considering stage of training', 1: 'Does not meet expected level of performance considering stage of training',
                         np.nan: 'No basis for evaluation'}

colorlegendfinal = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':'#4c72b0', 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':'#55a868',
                    'At expected level of performance considering stage of training: H3 (65-69)':'#e07a5f', 'Barely meets expected level of performance considering stage of training: P (50-64)':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training: N (0-49)':'#993f14', 'Clinical rotation not completed':'gray', 
                    'Far exceeds expected level of performance considering stage of training':'#4c72b0',
                    'Exceeds expected level of performance considering stage of training':'#55a868', 'At expected level of performance considering stage of training':'#e07a5f', 'Barely meets expected level of performance considering stage of training':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training':'#993f14', 'No basis for evaluation':'gray', 'Not applicable':'gray'}
mcgradedict = {'Completed with substantial assistance':1, 'Completed with minimal assistance':2, 'Completed independently (competent)':3, 'No basis for evaluation': np.nan}
generalgradedict = {'ALWAYS': 4, 'OFTEN': 3, 'SOMETIMES': 2, 'RARELY': 1, 'NEVER': 0, 'Not graded': np.nan}

# create color legend dictionary
colorlegendmc = {'Completed with substantial assistance': '#e07a5f', 'Completed with minimal assistance': '#55a868', 'Completed independently (competent)': '#4c72b0', 'No basis for evaluation': 'gray',
              }
colorlegendgeneral =  {'ALWAYS': '#4c72b0', 'OFTEN': '#55a868', 'SOMETIMES': '#e07a5f', 'RARELY': '#f9c74f', 'NEVER': '#993f14', 'Not graded': 'gray'}
reversemcgradedict = {v:k for k, v in mcgradedict.items()}
referencedf = pd.DataFrame(reversemcgradedict.items(), columns=['Grade', 'Description'])

# drop nan values from grade column
referencedf.dropna(inplace=True)
referencedf['Grade'] = referencedf['Grade'].astype(int)
referencedf.sort_values(by='Grade', inplace=True)
referencetable = createTable(referencedf, 'Score reference', [1, 5], customTextCols=[1])
df[mcCols] = df[mcCols].replace(mcgradedict)
df[colDate] = pd.to_datetime(df[colDate], format = 'mixed')
df.sort_values(by=[colName, colDate], inplace=True)
# take only may and june 2025 dates
# df = df[(df[colDate].dt.year == 2025) & (df[colDate].dt.month.isin([5, 6]))]
display(df.head())
df[colDate] = df[colDate].dt.date
df[colDate] = df[colDate].astype(str)
df[mcCols] = df[mcCols].astype(float)

# count the number of each scores for each student
df['#3'] = df[mcCols].apply(lambda x: x.value_counts().get(3, 0), axis=1)
df['#2'] = df[mcCols].apply(lambda x: x.value_counts().get(2, 0), axis=1)
df['#1'] = df[mcCols].apply(lambda x: x.value_counts().get(1, 0), axis=1)
df['#nan'] = df[mcCols].apply(lambda x: pd.isna(x)).sum(axis=1)

df[generalCols] = df[generalCols].replace(generalgradedict)
# display(df.head())

students = df[colName].unique()

def chunkList(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


def createPieCharts(mcCols, n, mcgradedict, colorlegend, elements, studentdf, fullnamedict):
    for groupIndex, mcChunk in enumerate(chunkList(mcCols, n)):
        fig, axes = plt.subplots(3, 2, figsize=(figSize[0], figSize[1]))
        fig.subplots_adjust(hspace=0.4, wspace=0.3)
        axes = axes.flatten()
        for idx, col in enumerate(mcChunk):
            ax = axes[idx]
            yTitle = 1.3
            faceColor = '#f2f2f2'  # background shading
            # add frame
            framePadding = 0.02
            bbox = ax.get_position()
            # Calculate extended box
            x0 = bbox.x0 - 2 * framePadding
            y0 = bbox.y0 - framePadding
            width = bbox.width + 4 * framePadding
            height = bbox.height + (yTitle - 1.23) + framePadding  # 1.0 is top of axes

            # Draw rectangle in figure space
            rect = Rectangle(
                (x0, y0), width, height,
                linewidth=1,
                edgecolor='black',
                facecolor=faceColor,
                transform=fig.transFigure,
                zorder=-5
            )
            fig.patches.append(rect)


            valuecounts = studentdf[col].value_counts(dropna=False)   
            display(valuecounts) 
            newkeys = []
            for v in valuecounts.index:
                for k, val in mcgradedict.items():
                    if (pd.isna(v) and pd.isna(val)) or (v == val):
                        newkeys.append(k)
                        break
            print(newkeys)

            valuecounts.index = newkeys
            total = valuecounts.sum()
            display(valuecounts)
            colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
            
            # create pie without labels
            wedges, _, autotexts = ax.pie(
                valuecounts,
                labels=None,
                autopct=lambda x: autopct(x, total),
                colors=colors,
                textprops={'fontsize': 7}
            )
            

            title = textwrap.fill(fullnamedict[col].split(' - ')[-1], width=55)  # Adjust width as needed
            # ax.set_title(title, fontsize=9)
            ax.text(0.5, yTitle, title, ha='center', va='top', fontsize=8, transform=ax.transAxes, wrap=True)


            # Add legend to bottom-left of figure
            legendHandles = [
                plt.Line2D([0], [0], marker='o', color='w', label=label,
                        markerfacecolor=color, markersize=8)
                for label, color in colorlegend.items()
            ]

            fig.legend(
                handles=legendHandles,
                loc='upper left',
                bbox_to_anchor=(0.09, 1.03),
                fontsize=8
            )

        # Hide unused subplots
        for j in range(len(mcChunk), n):
            axes[j].axis('off')


        # Save or display
        img = addPlotImage(fig, 0.9)
        # elements.append(KeepTogether([Paragraph(f"Batch {groupIndex+1} Pie Charts", smallsubsubheadingStyle), Spacer(1, 12), img]))
        elements.append(KeepTogether([img]))
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    # elements.append(PageBreak())
def getReportType2(student): # only plot the pie charts for each mc question, general question and final grade
    elements = []
    elements.append(Paragraph(f'{student}', style = subheadingStyle))
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 24))
    # plot mc question ignoring the nan values
    n = 6
    createPieCharts(mcCols, n, mcgradedict, colorlegendmc, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot general question ignoring the nan values
    createPieCharts(generalCols, n, generalgradedict, colorlegendgeneral, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    
    # create pie charts for extra columns similar to final grades--------------------------------------------------
    for col in extraCols:
        fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
        xy = studentdf[col].dropna()
        xy = xy.value_counts()
        total = xy.sum()
        xy = xy.sort_index()
        labels = [textwrap.fill(k, width=100) for k in xy.index]
        ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels)
        title = f'{col} Distribution'
        ax.set_title(None)
        img = addPlotImage(fig, 0.9)
        elements.append(KeepTogether([Paragraph(title, style = subheadingStyle), Spacer(1, 12), img]))
        plt.close(fig)
        elements.append(Spacer(1, 24))
    
    # add comments of Q10
    for col in commentCols:
        commentsDf = studentdf[['Staff', col]].dropna()
        commentsDf.rename(columns={col: 'Comments'}, inplace=True)
        display(commentsDf.head())
        
        table = createTable(
            commentsDf,
            f'{fullnamedict[col]}',
            [1, 5],
            customTextCols=[0, 1],
            titleStyle=variableUtils.smallsubsubheadingStyleC,
            tableTextStyle=tableTextStyleSmall
        )
        
        elements.append(table)
        elements.append(Spacer(1, 24))

    # add critical incident table
    if colCI in studentdf.columns and colCIComment in studentdf.columns:
        ciDf = studentdf[[colCI, colCIComment]].dropna()

        ciDf.rename(columns={colCI: 'Critical Incident', colCIComment: 'Details'}, inplace=True)
        ciDf = ciDf[ciDf['Critical Incident'] == 'Yes']  # filter only critical incidents
        if ciDf.empty:
            elements.append(Paragraph('No critical incidents reported.', style=tableTextStyleLarge))
            elements.append(Spacer(1, 24))
        else:
            display(ciDf.head())
            
            table = createTable(
                ciDf,
                f'{fullnamedict[colCI]}',
                [1, 5],
                customTextCols=[0, 1],
                titleStyle=variableUtils.smallsubsubheadingStyleC,
                tableTextStyle=tableTextStyleSmall
            )
            
            elements.append(table)
            elements.append(Spacer(1, 24))

    savefolder = f'{folder}/studentReports Brief'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)
students = df[colName].unique()
for student in students:
    getReportType2(student)
    # break
# getReport('Grace Wu')
# getReport('Andrew Park')

## Oral Med

In [ ]:
workbookpath = '2025\DCD\Oral Med\Postgraduate Clinical Evaluation - DCD in Oral Medicine_July 4, 2025_16.06.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
colFinished = 'Finished'
df = df[df[colFinished] == 'True']  # filter only finished forms
# mccols are with Q\d+_\d+ pattern
mcCols = [col for col in df.columns if re.match(r'Q\d+_\d+', col)]
mcColIdentifier = 'Q10'
# remove 
generalCols = [col for col in mcCols if mcColIdentifier in col]
mcCols = [col for col in mcCols if mcColIdentifier not in col]
mcCols = [col for col in mcCols if 'TEXT' not in col]  # remove columns with TEXT in the name
print(mcCols)
colDate = 'Date'

# comment columns are Q11 and Q12
commentCols =[ 'Q11', 'Q12']
finalgradedict = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':5, 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':4, 
                  'At expected level of performance considering stage of training: H3 (65-69)':3, 'Barely meets expected level of performance considering stage of training: P (50-64)':2,
                  'Does not meet expected level of performance considering stage of training: N (0-49)':1, 'No basis for evaluation':np.nan, 'Clinical rotation not completed':np.nan, 'Not applicable':np.nan,
                  'Exceeds expected level of performance considering stage of training': 4, 'At expected level of performance considering stage of training': 3, 'Barely meets expected level of performance considering stage of training':2}
reversefinalgradedict = {5: 'Far exceeds expected level of performance considering stage of training', 4: 'Exceeds expected level of performance considering stage of training',
                         3: 'At expected level of performance considering stage of training', 2: 'Barely meets expected level of performance considering stage of training', 1: 'Does not meet expected level of performance considering stage of training',
                         np.nan: 'No basis for evaluation'}

colorlegendfinal = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':'#4c72b0', 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':'#55a868',
                    'At expected level of performance considering stage of training: H3 (65-69)':'#e07a5f', 'Barely meets expected level of performance considering stage of training: P (50-64)':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training: N (0-49)':'#993f14', 'Clinical rotation not completed':'gray', 
                    'Far exceeds expected level of performance considering stage of training':'#4c72b0',
                    'Exceeds expected level of performance considering stage of training':'#55a868', 'At expected level of performance considering stage of training':'#e07a5f', 'Barely meets expected level of performance considering stage of training':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training':'#993f14', 'No basis for evaluation':'gray', 'Not applicable':'gray'}
colFinalGrade = 'Q22'
df[colFinalGrade] = df[colFinalGrade].replace(finalgradedict)
df[colFinalGrade] = df[colFinalGrade].astype(float)
mcgradedict = {'Completed with substantial assistance':1, 'Completed with minimal assistance':2, 'Completed independently (competent)':3, 'No basis for evaluation': np.nan}
generalgradedict = {'ALWAYS': 4, 'OFTEN': 3, 'SOMETIMES': 2, 'RARELY': 1, 'NEVER': 0, 'Not graded': np.nan}
# create color legend dictionary
colorlegendmc = {'Completed with substantial assistance': '#e07a5f', 'Completed with minimal assistance': '#55a868', 'Completed independently (competent)': '#4c72b0', 'No basis for evaluation': 'gray',
              }
colorlegendgeneral =  {'ALWAYS': '#4c72b0', 'OFTEN': '#55a868', 'SOMETIMES': '#e07a5f', 'RARELY': '#f9c74f', 'NEVER': '#993f14', 'Not graded': 'gray'}
reversemcgradedict = {v:k for k, v in mcgradedict.items()}
referencedf = pd.DataFrame(reversemcgradedict.items(), columns=['Grade', 'Description'])
# drop nan values from grade column
referencedf.dropna(inplace=True)
referencedf['Grade'] = referencedf['Grade'].astype(int)
referencedf.sort_values(by='Grade', inplace=True)
referencetable = createTable(referencedf, 'Score reference', [1, 5], customTextCols=[1])
df[mcCols] = df[mcCols].replace(mcgradedict)
df[colDate] = pd.to_datetime(df[colDate], format = 'mixed')
df.sort_values(by=[colName, colDate], inplace=True)
# take only may and june 2025 dates
# df = df[(df[colDate].dt.year == 2025) & (df[colDate].dt.month.isin([5, 6]))]
display(df.head())
df[colDate] = df[colDate].dt.date
df[colDate] = df[colDate].astype(str)
df[mcCols] = df[mcCols].astype(float)

# count the number of each scores for each student
df['#3'] = df[mcCols].apply(lambda x: x.value_counts().get(3, 0), axis=1)
df['#2'] = df[mcCols].apply(lambda x: x.value_counts().get(2, 0), axis=1)
df['#1'] = df[mcCols].apply(lambda x: x.value_counts().get(1, 0), axis=1)
df['#nan'] = df[mcCols].apply(lambda x: pd.isna(x)).sum(axis=1)

df[generalCols] = df[generalCols].replace(generalgradedict)
# display(df.head())

students = df[colName].unique()

def chunkList(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


def createPieCharts(mcCols, n, mcgradedict, colorlegend, elements, studentdf, fullnamedict):
    for groupIndex, mcChunk in enumerate(chunkList(mcCols, n)):
        fig, axes = plt.subplots(3, 2, figsize=(figSize[0], figSize[1]))
        fig.subplots_adjust(hspace=0.4, wspace=0.3)
        axes = axes.flatten()
        for idx, col in enumerate(mcChunk):
            ax = axes[idx]
            yTitle = 1.3
            faceColor = '#f2f2f2'  # background shading
            # add frame
            framePadding = 0.02
            bbox = ax.get_position()
            # Calculate extended box
            x0 = bbox.x0 - 2 * framePadding
            y0 = bbox.y0 - framePadding
            width = bbox.width + 4 * framePadding
            height = bbox.height + (yTitle - 1.23) + framePadding  # 1.0 is top of axes

            # Draw rectangle in figure space
            rect = Rectangle(
                (x0, y0), width, height,
                linewidth=1,
                edgecolor='black',
                facecolor=faceColor,
                transform=fig.transFigure,
                zorder=-5
            )
            fig.patches.append(rect)


            valuecounts = studentdf[col].value_counts(dropna=False)   
            display(valuecounts) 
            newkeys = []
            for v in valuecounts.index:
                for k, val in mcgradedict.items():
                    if (pd.isna(v) and pd.isna(val)) or (v == val):
                        newkeys.append(k)
                        break
            print(newkeys)

            valuecounts.index = newkeys
            total = valuecounts.sum()
            display(valuecounts)
            colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
            
            # create pie without labels
            wedges, _, autotexts = ax.pie(
                valuecounts,
                labels=None,
                autopct=lambda x: autopct(x, total),
                colors=colors,
                textprops={'fontsize': 7}
            )
            

            title = textwrap.fill(fullnamedict[col].split(' - ')[-1], width=55)  # Adjust width as needed
            # ax.set_title(title, fontsize=9)
            ax.text(0.5, yTitle, title, ha='center', va='top', fontsize=8, transform=ax.transAxes, wrap=True)


            # Add legend to bottom-left of figure
            legendHandles = [
                plt.Line2D([0], [0], marker='o', color='w', label=label,
                        markerfacecolor=color, markersize=8)
                for label, color in colorlegend.items()
            ]

            fig.legend(
                handles=legendHandles,
                loc='upper left',
                bbox_to_anchor=(0.09, 1.03),
                fontsize=8
            )

        # Hide unused subplots
        for j in range(len(mcChunk), n):
            axes[j].axis('off')


        # Save or display
        img = addPlotImage(fig, 0.9)
        # elements.append(KeepTogether([Paragraph(f"Batch {groupIndex+1} Pie Charts", smallsubsubheadingStyle), Spacer(1, 12), img]))
        elements.append(KeepTogether([img]))
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    # elements.append(PageBreak())
def getReportType2(student): # only plot the pie charts for each mc question, general question and final grade
    elements = []
    elements.append(Paragraph(f'{student}', style = subheadingStyle))
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 24))
    # plot mc question ignoring the nan values
    n = 6
    createPieCharts(mcCols, n, mcgradedict, colorlegendmc, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot general question ignoring the nan values
    createPieCharts(generalCols, n, generalgradedict, colorlegendgeneral, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot final grade ignoring the nan values ----------------------------------------------------------------------
    elements.append(Paragraph('Final Grades', style = subsubheadingStyle))
    elements.append(Spacer(1, 24))
    refdf2 = pd.DataFrame(reversefinalgradedict.items(), columns=['Grade', 'Description'])
    refdf2.dropna(inplace=True)
    refdf2['Grade'] = refdf2['Grade'].astype(int)
    # drop nan values from grade column

    refdf2.sort_values(by='Grade', inplace=True)
    refTable2 = createTable(refdf2, 'Final Grade Reference', [1, 5], customTextCols=[1])
    # elements.append(refTable2)
    # elements.append(Spacer(1, 24))
    # plot time series of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0], figSize[1]/5))
    xy = studentdf[[colDate, colFinalGrade]].dropna()
    xy = xy.groupby(colDate).mean().reset_index()     # aggregate the scores for each date by taking the mean
    ax.plot(xy[colDate], xy[colFinalGrade])
    title = 'Average Time Series'
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=6)
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))     # integer ticks on y axis
    ax.set_ylim(0, 5.4)
    ax.set_title(title, fontsize=10)
    plt.grid('--', alpha=0.5)
    img = addPlotImage(fig, 0.9)
    # elements.append(img)
    plt.close(fig)
    # elements.append(Spacer(1, 24))

    # pie chart of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    xy = studentdf[colFinalGrade].dropna()
    xy = xy.value_counts()
    total = xy.sum()
    xy = xy.sort_index()
    xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
    labels = [textwrap.fill(k, width=100) for k in xy.index]
    ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels, colors=[colorlegendfinal.get(k, 'gray') for k in xy.index])
    # Add legend to bottom-left of figure
    # legendHandles = [
    #     plt.Line2D([0], [0], marker='o', color='w', label=label,
    #             markerfacecolor=color, markersize=6)
    #     for label, color in colorlegendfinal.items()
    # ]

    # fig.legend(
    #     handles=legendHandles,
    #     loc='upper left',
    #     bbox_to_anchor=(0.09, 0.03),
    #     fontsize=6
    # )
    title = 'Final Grade Distribution'
    ax.set_title(None)
    img = addPlotImage(fig, 0.9)
    elements.append(img)
    plt.close(fig)
    elements.append(Spacer(1, 24))
    
    # add comments of Q20 and Q21 as table
    for col in commentCols:
        commentsDf = studentdf[['Staff', col]].dropna()
        commentsDf.rename(columns={col: 'Comments'}, inplace=True)
        display(commentsDf.head())
        
        table = createTable(
            commentsDf,
            f'{fullnamedict[col]}',
            [1, 5],
            customTextCols=[0, 1],
            titleStyle=variableUtils.smallsubsubheadingStyleC,
            tableTextStyle=tableTextStyleSmall
        )
        
        elements.append(table)
        elements.append(Spacer(1, 24))
    # elements.append(Spacer(1, 24))

    savefolder = f'{folder}/studentReports Brief'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)
students = df[colName].unique()
for student in students:
    getReportType2(student)
    # break
# getReport('Grace Wu')
# getReport('Andrew Park')

## Endo

In [50]:
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_CENTER
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

# Register Arial if available
pdfmetrics.registerFont(TTFont('Arial', 'arial.ttf'))

bannerHeadingStyle = ParagraphStyle(
    name='BannerHeading',
    fontName='Helvetica-Bold',  # ensures actual bold
    fontSize=18,
    textColor='white',
    alignment=0,
    leading=22,
    spaceAfter=0,
    spaceBefore=0
)
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

pdfmetrics.registerFont(TTFont('Calibri-Bold', 'calibrib.ttf'))  # Bold version



def getBannerDrawer(name):
    def drawBanner(canvas, doc):
        canvas.saveState()

        # Banner layout
        pageWidth, pageHeight = doc.pagesize
        bannerHeight = 132
        canvas.setFillColor(colors.HexColor("#010d44"))
        canvas.rect(0, pageHeight - bannerHeight, pageWidth, bannerHeight, fill=1, stroke=0)

        # Text: internal margin from left and top
        textLeftMargin = leftMargin
        topOffset = 72  # Distance from top of banner to first text line
        lineSpacing = 36

        try:
            canvas.setFont("Calibri-Bold", 30)
        except:
            canvas.setFont("Helvetica-Bold", 30)  # Fallback

        canvas.setFillColor(colors.white)
        canvas.drawString(textLeftMargin, pageHeight - topOffset, "DCD Endodontics")
        try:
            canvas.setFont("Calibri-Bold", 24)
        except:
            canvas.setFont("Helvetica-Bold", 24)  # Fallback        
        
        canvas.drawString(textLeftMargin, pageHeight - topOffset - lineSpacing, f"{name}")

        canvas.restoreState()

    return drawBanner

In [51]:
workbookpath = '2025\DCD\Endo\Postgraduate Clinical Evaluation - DCD in Endodontics_July 29, 2025_09.17.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
colFinished = 'Finished'
df = df[df[colFinished] == 'True']  # filter only finished forms

mcCols= [
    "ProfessionalismQ7",
    "Dialogue",
    "AssessmentQ229",
    "Diff Diagnosis",
    "Treatment Plan",
    "Intervention",
    "Evaluation"
]

print(mcCols)
colDate = 'Date'

# comment columns are Q11 and Q12
colOverallAssessment = 'Q234'
commentCols =[ 'Q11', 'Q12']
colFinalGrade = 'Q22'

students = df[colName].unique()
for student in students:
    elements = []
    # elements.append(Paragraph(f'{student}', style = subheadingStyle))
    # elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    # elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 72))
    idx = 0
    for i, row in studentdf.iterrows():
        idx+=1
        elements.append(Paragraph(f'Form {idx} ({row[colDate]})', style = subsubheadingStyle))
        elements.append(Spacer(1, 12))
        # create table
        # create table for mcCols
        evalDf = pd.DataFrame(columns=['Question', 'Evaluation'])
        for col in mcCols:
            newRow = {
                    'Question': fullnamedict[col].split('. ')[-1], #.replace('\n', '<br/>'),
                    'Evaluation': row[col]
                }
            evalDf = pd.concat([evalDf, pd.DataFrame([newRow])], ignore_index=True)
        evalTable = createTable(
            evalDf,
            'Evaluation',
            [5, 2.5],
            customTextCols=[0, 1],
            titleStyle=variableUtils.smallsubsubheadingStyleC,
            tableTextStyle=tableTextStyleSmall,
            # headerColor='#bebebe'
        )
        elements.append(evalTable)
        elements.append(Spacer(1, 24))
        # add overall assessment
        overallAssessment = row[colOverallAssessment]
        if pd.isna(overallAssessment):
            overallAssessment = 'No overall assessment provided.'
        else:
            overallAssessment = overallAssessment.replace('\n', '<br/>')
        colOverallText = fullnamedict[colOverallAssessment].replace('\n', '</br>')
        elements.append(Paragraph(f'{colOverallText}', style=variableUtils.smallsubsubheadingStyleL))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph(overallAssessment, style=tableTextStyleL))
        elements.append(Spacer(1, 24))
        # add comments of Q11 and Q12 
        for col in commentCols:
            elements.append(Paragraph(f'{fullnamedict[col]}', style=variableUtils.smallsubsubheadingStyleL))
            elements.append(Spacer(1, 12))
            if pd.isna(row[col]):
                elements.append(Paragraph('No comments provided.', style=tableTextStyleL))
            else:
                elements.append(Paragraph(row[col], style=tableTextStyleL))
        
            elements.append(Spacer(1, 24))

        # add final grade
        elements.append(Paragraph('Final Grade', style=variableUtils.smallsubsubheadingStyleL))
        elements.append(Spacer(1, 12))
        finalGrade = row[colFinalGrade]
        if pd.isna(finalGrade):
            elements.append(Paragraph('No final grade provided.', style=tableTextStyleL))
        else:
            elements.append(Paragraph(f'{row[colFinalGrade]}', style=tableTextStyleL))

        elements.append(PageBreak())
    
    os.makedirs(f'{folder}/studentReports', exist_ok=True)
    doc = SimpleDocTemplate(f'{folder}/studentReports/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, 
                            leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    # add overall assessment
    doc.build(elements, onFirstPage=getBannerDrawer(student))


['ProfessionalismQ7', 'Dialogue', 'AssessmentQ229', 'Diff Diagnosis', 'Treatment Plan', 'Intervention', 'Evaluation']


,Finished,RecordedDate,ResponseId,Date,Student Name,Semester,Site,Staff,ProfessionalismQ7,Dialogue,AssessmentQ229,Diff Diagnosis,Treatment Plan,Intervention,Evaluation,Q234,Q11,Q12,Q22,Q23_Id
1,True,2025-05-29 15:48:13.833000,R_4szhmgw2JmixUqF,29 May 2025,Laura Raguine,1,Royal Dental Hospital Melbourne RDHM,Satish alapati,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Progressing appropriately,Taking radiographs for assessment of canal ori...,NaN,Exceeds expected level of performance consider...,F_2diRq8e5drdrGPw
13,True,2025-07-04 12:16:33.115000,R_4Pmja870iagVEad,4 Jul 2025,Laura Raguine,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Surpassing expectations at this point in educa...,Good patient management,None at this session,Exceeds expected level of performance consider...,F_25ZSJCkIqpLUQgp
16,True,2025-07-11 11:25:19.845000,R_4wB2oM0hquztKif,11 Jul 2025,Laura Raguine,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Progressing appropriately,Good diagnosis and patient management,Need to increase speed and efficiency,Exceeds expected level of performance consider...,F_2XhvyzVvJI3Q3ig


Creating table for Evaluation
Creating table for Evaluation
Creating table for Evaluation


,Finished,RecordedDate,ResponseId,Date,Student Name,Semester,Site,Staff,ProfessionalismQ7,Dialogue,AssessmentQ229,Diff Diagnosis,Treatment Plan,Intervention,Evaluation,Q234,Q11,Q12,Q22,Q23_Id
2,True,2025-05-29 15:50:41.837000,R_4CwqyIygd5GWKzh,29 May 2025,Yoshikatsu Shibata,1,Royal Dental Hospital Melbourne RDHM,Satish Alapati,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Progressing appropriately,Case management aspects and choices of appropr...,NaN,Exceeds expected level of performance consider...,F_Y318hIGjlPcAc1z
9,True,2025-07-03 12:32:06.911000,R_4wrRhbfvGKrStwt,3 Jul 2025,Yoshikatsu Shibata,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Progressing appropriately,Over all good patient management,None at this time,Exceeds expected level of performance consider...,F_2Eie7YD7dWwJbq2
12,True,2025-07-04 12:15:05.623000,R_4zPeYi2LLOBQpSw,4 Jul 2025,Yoshikatsu Shibata,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Surpassing expectations at this point in educa...,Good clinical judgment,None at this time,Exceeds expected level of performance consider...,F_1hGMvjF4XqIvkHc
15,True,2025-07-11 11:21:02.038000,R_4f7qE9vAsJBWUNu,11 Jul 2025,Yoshikatsu Shibata,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Critically behind expectations at this point i...,Good re-treatment case with good WL radiographs,None at this time,Exceeds expected level of performance consider...,F_21mPo0wAl2IT02D


Creating table for Evaluation
Creating table for Evaluation
Creating table for Evaluation
Creating table for Evaluation


...

Creating table for Evaluation
Creating table for Evaluation


,Finished,RecordedDate,ResponseId,Date,Student Name,Semester,Site,Staff,ProfessionalismQ7,Dialogue,AssessmentQ229,Diff Diagnosis,Treatment Plan,Intervention,Evaluation,Q234,Q11,Q12,Q22,Q23_Id
8,True,2025-07-03 12:30:19.386000,R_9GK6mJw7n2fbKiV,3 Jul 2025,Scott Murphy,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Critically behind expectations at this point i...,Overall good patient management,Need time management,Exceeds expected level of performance consider...,F_2wazt8FpqFRojWS


Creating table for Evaluation


,Finished,RecordedDate,ResponseId,Date,Student Name,Semester,Site,Staff,ProfessionalismQ7,Dialogue,AssessmentQ229,Diff Diagnosis,Treatment Plan,Intervention,Evaluation,Q234,Q11,Q12,Q22,Q23_Id
10,True,2025-07-03 12:33:30.729000,R_9NCGgr075Jg5FKB,3 Jul 2025,Angus Marshall,2,Royal Dental Hospital Melbourne RDHM,NaN,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Meets expectations at this point in education,Progressing appropriately,Good patient management skills,None at this time,Exceeds expected level of performance consider...,F_UroNIKWXdVUNOKd


Creating table for Evaluation


In [ ]:
workbookpath = '2025\DCD\Endo\Postgraduate Clinical Evaluation - DCD in Endodontics_July 29, 2025_09.17.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
colFinished = 'Finished'
df = df[df[colFinished] == 'True']  # filter only finished forms

mcCols= [
    "ProfessionalismQ7",
    "Dialogue",
    "AssessmentQ229",
    "Diff Diagnosis",
    "Treatment Plan",
    "Intervention",
    "Evaluation"
]

print(mcCols)
colDate = 'Date'

# comment columns are Q11 and Q12
colOverallAssessment = 'Q234'
commentCols =[ 'Q11', 'Q12']
finalgradedict = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':5, 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':4, 
                  'At expected level of performance considering stage of training: H3 (65-69)':3, 'Barely meets expected level of performance considering stage of training: P (50-64)':2,
                  'Does not meet expected level of performance considering stage of training: N (0-49)':1, 'No basis for evaluation':np.nan, 'Clinical rotation not completed':np.nan, 'Not applicable':np.nan,
                  'Exceeds expected level of performance considering stage of training': 4, 'At expected level of performance considering stage of training': 3, 'Barely meets expected level of performance considering stage of training':2}
reversefinalgradedict = {5: 'Far exceeds expected level of performance considering stage of training', 4: 'Exceeds expected level of performance considering stage of training',
                         3: 'At expected level of performance considering stage of training', 2: 'Barely meets expected level of performance considering stage of training', 1: 'Does not meet expected level of performance considering stage of training',
                         np.nan: 'No basis for evaluation'}

colorlegendfinal = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':'#4c72b0', 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':'#55a868',
                    'At expected level of performance considering stage of training: H3 (65-69)':'#e07a5f', 'Barely meets expected level of performance considering stage of training: P (50-64)':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training: N (0-49)':'#993f14', 'Clinical rotation not completed':'gray', 
                    'Far exceeds expected level of performance considering stage of training':'#4c72b0',
                    'Exceeds expected level of performance considering stage of training':'#55a868', 'At expected level of performance considering stage of training':'#e07a5f', 'Barely meets expected level of performance considering stage of training':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training':'#993f14', 'No basis for evaluation':'gray', 'Not applicable':'gray'}
colFinalGrade = 'Q22'
df[colFinalGrade] = df[colFinalGrade].replace(finalgradedict)
df[colFinalGrade] = df[colFinalGrade].astype(float)
mcgradedict = {'Completed with substantial assistance':1, 'Completed with minimal assistance':2, 'Completed independently (competent)':3, 'No basis for evaluation': np.nan}
generalgradedict = {'ALWAYS': 4, 'OFTEN': 3, 'SOMETIMES': 2, 'RARELY': 1, 'NEVER': 0, 'Not graded': np.nan}
# create color legend dictionary
colorlegendmc = {'Completed with substantial assistance': '#e07a5f', 'Completed with minimal assistance': '#55a868', 'Completed independently (competent)': '#4c72b0', 'No basis for evaluation': 'gray',
              }
colorlegendgeneral =  {'ALWAYS': '#4c72b0', 'OFTEN': '#55a868', 'SOMETIMES': '#e07a5f', 'RARELY': '#f9c74f', 'NEVER': '#993f14', 'Not graded': 'gray'}
reversemcgradedict = {v:k for k, v in mcgradedict.items()}
referencedf = pd.DataFrame(reversemcgradedict.items(), columns=['Grade', 'Description'])
# drop nan values from grade column
referencedf.dropna(inplace=True)
referencedf['Grade'] = referencedf['Grade'].astype(int)
referencedf.sort_values(by='Grade', inplace=True)
referencetable = createTable(referencedf, 'Score reference', [1, 5], customTextCols=[1])
df[mcCols] = df[mcCols].replace(mcgradedict)
df[colDate] = pd.to_datetime(df[colDate], format = 'mixed')
df.sort_values(by=[colName, colDate], inplace=True)
# take only may and june 2025 dates
# df = df[(df[colDate].dt.year == 2025) & (df[colDate].dt.month.isin([5, 6]))]
display(df.head())
df[colDate] = df[colDate].dt.date
df[colDate] = df[colDate].astype(str)
df[mcCols] = df[mcCols].astype(float)

# count the number of each scores for each student
df['#3'] = df[mcCols].apply(lambda x: x.value_counts().get(3, 0), axis=1)
df['#2'] = df[mcCols].apply(lambda x: x.value_counts().get(2, 0), axis=1)
df['#1'] = df[mcCols].apply(lambda x: x.value_counts().get(1, 0), axis=1)
df['#nan'] = df[mcCols].apply(lambda x: pd.isna(x)).sum(axis=1)

df[generalCols] = df[generalCols].replace(generalgradedict)
# display(df.head())

students = df[colName].unique()

def chunkList(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


def createPieCharts(mcCols, n, mcgradedict, colorlegend, elements, studentdf, fullnamedict):
    for groupIndex, mcChunk in enumerate(chunkList(mcCols, n)):
        fig, axes = plt.subplots(3, 2, figsize=(figSize[0], figSize[1]))
        fig.subplots_adjust(hspace=0.4, wspace=0.3)
        axes = axes.flatten()
        for idx, col in enumerate(mcChunk):
            ax = axes[idx]
            yTitle = 1.3
            faceColor = '#f2f2f2'  # background shading
            # add frame
            framePadding = 0.02
            bbox = ax.get_position()
            # Calculate extended box
            x0 = bbox.x0 - 2 * framePadding
            y0 = bbox.y0 - framePadding
            width = bbox.width + 4 * framePadding
            height = bbox.height + (yTitle - 1.23) + framePadding  # 1.0 is top of axes

            # Draw rectangle in figure space
            rect = Rectangle(
                (x0, y0), width, height,
                linewidth=1,
                edgecolor='black',
                facecolor=faceColor,
                transform=fig.transFigure,
                zorder=-5
            )
            fig.patches.append(rect)


            valuecounts = studentdf[col].value_counts(dropna=False)   
            display(valuecounts) 
            newkeys = []
            for v in valuecounts.index:
                for k, val in mcgradedict.items():
                    if (pd.isna(v) and pd.isna(val)) or (v == val):
                        newkeys.append(k)
                        break
            print(newkeys)

            valuecounts.index = newkeys
            total = valuecounts.sum()
            display(valuecounts)
            colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
            
            # create pie without labels
            wedges, _, autotexts = ax.pie(
                valuecounts,
                labels=None,
                autopct=lambda x: autopct(x, total),
                colors=colors,
                textprops={'fontsize': 7}
            )
            

            title = textwrap.fill(fullnamedict[col].split(' - ')[-1], width=55)  # Adjust width as needed
            # ax.set_title(title, fontsize=9)
            ax.text(0.5, yTitle, title, ha='center', va='top', fontsize=8, transform=ax.transAxes, wrap=True)


            # Add legend to bottom-left of figure
            legendHandles = [
                plt.Line2D([0], [0], marker='o', color='w', label=label,
                        markerfacecolor=color, markersize=8)
                for label, color in colorlegend.items()
            ]

            fig.legend(
                handles=legendHandles,
                loc='upper left',
                bbox_to_anchor=(0.09, 1.03),
                fontsize=8
            )

        # Hide unused subplots
        for j in range(len(mcChunk), n):
            axes[j].axis('off')


        # Save or display
        img = addPlotImage(fig, 0.9)
        # elements.append(KeepTogether([Paragraph(f"Batch {groupIndex+1} Pie Charts", smallsubsubheadingStyle), Spacer(1, 12), img]))
        elements.append(KeepTogether([img]))
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    # elements.append(PageBreak())
def getReportType2(student): # only plot the pie charts for each mc question, general question and final grade
    elements = []
    elements.append(Paragraph(f'{student}', style = subheadingStyle))
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 24))
    # plot mc question ignoring the nan values
    n = 6
    createPieCharts(mcCols, n, mcgradedict, colorlegendmc, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot general question ignoring the nan values
    createPieCharts(generalCols, n, generalgradedict, colorlegendgeneral, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot final grade ignoring the nan values ----------------------------------------------------------------------
    elements.append(Paragraph('Final Grades', style = subsubheadingStyle))
    elements.append(Spacer(1, 24))
    refdf2 = pd.DataFrame(reversefinalgradedict.items(), columns=['Grade', 'Description'])
    refdf2.dropna(inplace=True)
    refdf2['Grade'] = refdf2['Grade'].astype(int)
    # drop nan values from grade column

    refdf2.sort_values(by='Grade', inplace=True)
    refTable2 = createTable(refdf2, 'Final Grade Reference', [1, 5], customTextCols=[1])
    # elements.append(refTable2)
    # elements.append(Spacer(1, 24))
    # plot time series of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0], figSize[1]/5))
    xy = studentdf[[colDate, colFinalGrade]].dropna()
    xy = xy.groupby(colDate).mean().reset_index()     # aggregate the scores for each date by taking the mean
    ax.plot(xy[colDate], xy[colFinalGrade])
    title = 'Average Time Series'
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=6)
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))     # integer ticks on y axis
    ax.set_ylim(0, 5.4)
    ax.set_title(title, fontsize=10)
    plt.grid('--', alpha=0.5)
    img = addPlotImage(fig, 0.9)
    # elements.append(img)
    plt.close(fig)
    # elements.append(Spacer(1, 24))

    # pie chart of final grades
    fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    xy = studentdf[colFinalGrade].dropna()
    xy = xy.value_counts()
    total = xy.sum()
    xy = xy.sort_index()
    xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
    labels = [textwrap.fill(k, width=100) for k in xy.index]
    ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels, colors=[colorlegendfinal.get(k, 'gray') for k in xy.index])
    # Add legend to bottom-left of figure
    # legendHandles = [
    #     plt.Line2D([0], [0], marker='o', color='w', label=label,
    #             markerfacecolor=color, markersize=6)
    #     for label, color in colorlegendfinal.items()
    # ]

    # fig.legend(
    #     handles=legendHandles,
    #     loc='upper left',
    #     bbox_to_anchor=(0.09, 0.03),
    #     fontsize=6
    # )
    title = 'Final Grade Distribution'
    ax.set_title(None)
    img = addPlotImage(fig, 0.9)
    elements.append(img)
    plt.close(fig)
    elements.append(Spacer(1, 24))
    
    # add comments of Q20 and Q21 as table
    for col in commentCols:
        commentsDf = studentdf[['Staff', col]].dropna()
        commentsDf.rename(columns={col: 'Comments'}, inplace=True)
        display(commentsDf.head())
        
        table = createTable(
            commentsDf,
            f'{fullnamedict[col]}',
            [1, 5],
            customTextCols=[0, 1],
            titleStyle=variableUtils.smallsubsubheadingStyleC,
            tableTextStyle=tableTextStyleSmall
        )
        
        elements.append(table)
        elements.append(Spacer(1, 24))
    # elements.append(Spacer(1, 24))

    savefolder = f'{folder}/studentReports Brief'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)
students = df[colName].unique()
for student in students:
    getReportType2(student)
    # break
# getReport('Grace Wu')
# getReport('Andrew Park')